# Simplex piecewise-linear underestimator with user-chosen interpolation points (P0 case)

This notebook lets you **freely specify interpolation points** (per scenario or shared).  
Default points: \(y \in \{0,\,0.7,\,1.5,\,2.4,\,3\}\).

For each scenario \(\omega\):
1. Build the piecewise-linear interpolant \(A_\omega(y)\) through the chosen points.
2. Compute \(m_\omega = \min_{y\in[0,3]}\{Q_\omega(y)-A_\omega(y)\}\).
3. Use \(A_\omega(y)+m_\omega\) as a valid underestimator on the grid.

Outputs (same figure sizing as your panel-size version):
1. Scenario 1: \(A_1\) and \(v_1\)
2. Scenario 2: \(A_2\) and \(v_2\)
3. Scenario 1: \(A_1+m_1\) and \(v_1\)
4. Scenario 2: \(A_2+m_2\) and \(v_2\)
5. Total: \(v\) and \((A_1+m_1)+(A_2+m_2)\)

Solver: Pyomo + **Gurobi** if available (falls back to another LP solver if not).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyomo.environ as pyo

# --------------------------
# Match plotting style from your CZ/LG notebook
# --------------------------
plt.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "cm",
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "axes.linewidth": 1.0,
})

# Panel sizing to match your "panelSize" notebook
FIGSIZE_PANEL = (5.1, 3.6)   # approx size of a single panel (a)/(b)
FIGSIZE_WIDE  = (10.2, 3.6)  # approx size of wide bottom panel (c)

# Domain
Y_L, Y_U = 0.0, 3.0
h = 0.001
ys = np.arange(Y_L, Y_U + 1e-12, h)
N = len(ys)
print("Grid points:", N, " step=", h)

def pick_solver():
    for name in ["gurobi", "gurobi_direct", "gurobi_persistent", "cbc", "glpk", "highs"]:
        try:
            opt = pyo.SolverFactory(name)
            if opt is not None and opt.available(exception_flag=False):
                print("Using solver:", name)
                return opt, name
        except Exception:
            continue
    raise RuntimeError("No LP/MIP solver found. Install Gurobi (recommended) or GLPK/CBC/HiGHS.")

opt, solver_name = pick_solver()


In [ ]:
# --------------------------
# P0 scenario value functions (Appendix C, P0-V)
# --------------------------
def Q1(y):
    y = np.asarray(y)
    return 1.00694*y**3 - 4.74589*y**2 + 5.17523*y

def Q2(y):
    y = np.asarray(y)
    return -0.677232*y**3 + 3.03949*y**2 - 3.02338*y

def Qtot(y):
    return Q1(y) + Q2(y)

Q1_vals = Q1(ys)
Q2_vals = Q2(ys)
Qtot_vals = Q1_vals + Q2_vals


In [ ]:
# --------------------------
# Choose interpolation points HERE
# - Default points: [0, 0.7, 1.5, 2.4, 3]
# - You can set different points for each scenario if you want.
# --------------------------
x_pts_s1 = [0.0, 0.7, 1.5, 2.4, 3.0]
x_pts_s2 = [0.0, 0.7, 1.5, 2.4, 3.0]

# Safety: sort and clip to [0,3]
def sanitize_pts(x_pts):
    x = sorted(set(float(t) for t in x_pts))
    if x[0] < Y_L - 1e-12 or x[-1] > Y_U + 1e-12:
        raise ValueError("Interpolation points must lie in [0,3].")
    if abs(x[0] - Y_L) > 1e-12 or abs(x[-1] - Y_U) > 1e-12:
        raise ValueError("Include both endpoints 0 and 3 to match the method description.")
    return x

x_pts_s1 = sanitize_pts(x_pts_s1)
x_pts_s2 = sanitize_pts(x_pts_s2)

print("Scenario 1 pts:", x_pts_s1)
print("Scenario 2 pts:", x_pts_s2)


In [ ]:
def piecewise_linear_interp(x_pts, y_pts, x_query):
    # Piecewise-linear interpolant through (x_pts, y_pts), evaluated at x_query. x_pts sorted.
    return np.interp(x_query, x_pts, y_pts)

def compute_ms_via_lp(diff_vals, solver):
    # Compute ms = min_i diff_vals[i] via LP:
    #   maximize ms
    #   s.t. ms <= diff_i  for all i
    m = pyo.ConcreteModel()
    m.ms = pyo.Var(domain=pyo.Reals)
    m.I = pyo.RangeSet(0, len(diff_vals)-1)

    def c_rule(mm, i):
        return mm.ms <= float(diff_vals[i])
    m.c = pyo.Constraint(m.I, rule=c_rule)

    m.obj = pyo.Objective(expr=m.ms, sense=pyo.maximize)
    solver.solve(m, tee=False)
    return float(pyo.value(m.ms))

def build_As_and_shift(Q_vals, x_pts):
    # Values at interpolation points (exact from function values on grid via interp)
    y_pts = [float(np.interp(x, ys, Q_vals)) for x in x_pts]
    As_vals = piecewise_linear_interp(np.array(x_pts), np.array(y_pts), ys)
    diff = Q_vals - As_vals
    ms = compute_ms_via_lp(diff, opt)
    As_shift = As_vals + ms
    y_m = float(ys[int(np.argmin(diff))])
    return As_vals, ms, As_shift, y_m, float(diff.min())

As1, ms1, As1_shift, y_m1, mind1 = build_As_and_shift(Q1_vals, x_pts_s1)
As2, ms2, As2_shift, y_m2, mind2 = build_As_and_shift(Q2_vals, x_pts_s2)

print("Scenario 1: ms1 =", ms1, " min(diff) =", mind1, " argmin y =", y_m1)
print("Scenario 2: ms2 =", ms2, " min(diff) =", mind2, " argmin y =", y_m2)

sum_shift = As1_shift + As2_shift


In [ ]:
# --------------------------
# Plot helpers (match your style)
# Adjust line widths here:
# --------------------------
LW_TRUE  = 2.0
LW_AS    = 2.0
LW_SHIFT = 2.0

def save_or_show(fig, savepath):
    if savepath:
        fig.savefig(savepath, dpi=300, bbox_inches="tight")
    return fig

def plot_Q_vs_As(Qvals, Asvals, ylabel, color_Q, color_As, label_Q, label_As, figsize, savepath=None):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    ax.plot(ys, Qvals, color=color_Q, lw=LW_TRUE, label=label_Q)
    ax.plot(ys, Asvals, color=color_As, lw=LW_AS, ls="--", label=label_As)
    ax.set_xlabel(r"$y$")
    ax.set_ylabel(ylabel)
    ax.set_xlim(Y_L, Y_U)
    ax.legend(loc="upper right", frameon=True)
    return save_or_show(fig, savepath)

def plot_Q_vs_Shift(Qvals, Shiftvals, ylabel, color_Q, color_Shift, label_Q, label_Shift, figsize, savepath=None):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    ax.plot(ys, Qvals, color=color_Q, lw=LW_TRUE, label=label_Q)
    ax.plot(ys, Shiftvals, color=color_Shift, lw=LW_SHIFT, ls="--", label=label_Shift)
    ax.set_xlabel(r"$y$")
    ax.set_ylabel(ylabel)
    ax.set_xlim(Y_L, Y_U)
    ax.legend(loc="upper right", frameon=True)
    return save_or_show(fig, savepath)

def plot_total(Qtot, sumShift, figsize, savepath=None):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    ax.plot(ys, Qtot, color="black", lw=LW_TRUE, label=r"$v$")
    ax.plot(ys, sumShift, color="black", lw=LW_SHIFT, ls="--",
            label=r"$(A_1+m_1) + (A_2+m_2)$")
    ax.set_xlabel(r"$y$")
    ax.set_ylabel(r"$v(y)$")
    ax.set_xlim(Y_L, Y_U)
    ax.legend(loc="upper right", frameon=True)
    return save_or_show(fig, savepath)


In [ ]:
# --------------------------
# Output 5 figures (as requested)
# --------------------------

# 1) Scenario 1: As and true
plot_Q_vs_As(
    Q1_vals, As1,
    ylabel=r"$v_1(y)$",
    color_Q="red", color_As="black",
    label_Q=r"$v_1$", label_As=r"$A_1$",
    figsize=FIGSIZE_PANEL,
    savepath="fixedpts_fig1_s1_As.png"
)
plt.show()

# 2) Scenario 2: As and true
plot_Q_vs_As(
    Q2_vals, As2,
    ylabel=r"$v_2(y)$",
    color_Q="deepskyblue", color_As="black",
    label_Q=r"$v_2$", label_As=r"$A_2$",
    figsize=FIGSIZE_PANEL,
    savepath="fixedpts_fig2_s2_As.png"
)
plt.show()

# 3) Scenario 1: As+ms and true
plot_Q_vs_Shift(
    Q1_vals, As1_shift,
    ylabel=r"$v_1(y)$",
    color_Q="red", color_Shift="black",
    label_Q=r"$v_1$", label_Shift=r"$A_1+m_1$",
    figsize=FIGSIZE_PANEL,
    savepath="fixedpts_fig3_s1_As_ms.png"
)
plt.show()

# 4) Scenario 2: As+ms and true
plot_Q_vs_Shift(
    Q2_vals, As2_shift,
    ylabel=r"$v_2(y)$",
    color_Q="deepskyblue", color_Shift="black",
    label_Q=r"$v_2$", label_Shift=r"$A_2+m_2$",
    figsize=FIGSIZE_PANEL,
    savepath="fixedpts_fig4_s2_As_ms.png"
)
plt.show()

# 5) Total: true v and sum of (As+ms)
plot_total(
    Qtot_vals, sum_shift,
    figsize=FIGSIZE_WIDE,
    savepath="fixedpts_fig5_total.png"
)
plt.show()

print("Saved PNGs:")
print("  fixedpts_fig1_s1_As.png")
print("  fixedpts_fig2_s2_As.png")
print("  fixedpts_fig3_s1_As_ms.png")
print("  fixedpts_fig4_s2_As_ms.png")
print("  fixedpts_fig5_total.png")
